In [1]:
!pip install requests beautifulsoup4 tqdm
!pip install selenium webdriver-manager


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [23]:
import numpy as np
import pandas as pd
import os
import time

ds = pd.read_csv("./data/sneakers2023.csv")

In [24]:
#create folder to store image data
img_folder = "feetflix"
os.makedirs(img_folder, exist_ok=True)

In [ ]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument("--no-sandbox")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# scrape stockx website images
def img_scraping(shoe_name):
    search_url = f"https://stockx.com/search?s={shoe_name.replace(' ', '+')}"
    
    try:
        driver.get(search_url)
        time.sleep(3)
    
        img_element = driver.find_element(By.TAG_NAME, "img")
        img_url = img_element.get_attribute("src")
        return img_url
    except Exception as e:
        print(f"Error scraping {shoe_name}: {e}")
        return None

# searching stockx iteratively based on unique item name    
for shoe_name in ds["item"].unique():
        img_url = img_scraping(shoe_name)
        
        if img_url:
            try:
                img_data = requests.get(img_url).content
                img_path = os.path.join(img_folder, f"{shoe_name.replace('/', '-')}.jpg")
                
                with open(img_path, "wb") as img_file:
                    img_file.write(img_data)
                print(f"Saved: {shoe_name}.jpg")
            except Exception as e:
                print(f"Error saving {shoe_name}: {e}")
        else:
            print(f"Image not found: {shoe_name}")
        time.sleep(2)
driver.quit()

In [25]:
# insert new column img_path to dataset
if os.path.exists(img_folder):
    img_path = {}
    
    for filename in os.listdir(img_folder):
        shoe_name = filename.replace(".jpg", "")
        img_path[shoe_name] = os.path.join(img_folder, filename)
        
    ds["img_path"] = ds["item"].str.replace("/", "-").map(img_path)
    print(ds.head(10))
else:
    print("No images found in the folder.")

   Unnamed: 0                                               item        brand  \
0           0                       Jordan 4 Retro SB Pine Green       Jordan   
1           1             Jordan 3 Retro White Cement Reimagined       Jordan   
2           2            Nike Air Force 1 Low Tiffany & Co. 1837         Nike   
3           3            Nike Air Max 1 '86 Big Bubble Sport Red         Nike   
4           4                                 MSCHF Big Red Boot        MSCHF   
5           5           New Balance 990v6 Action Bronson Baklava  New Balance   
6           6       Nike Dunk Low Retro White Black Panda (2021)         Nike   
7           7  Nike Dunk Low Retro White Black Panda (2021) (...         Nike   
8           8                             Nike Dunk Low Grey Fog         Nike   
9           9  Nike Dunk Low Retro White Black Panda (2021) (GS)         Nike   

   retail     release  lowestAsk  numberOfAsks  salesThisPeriod  highestBid  \
0     225  2023-03-21        

In [30]:
ds.to_csv("cleaned_sneaks.csv", index=False)